# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План проекта**:
- определить и заполнить пропущенные значения:
- описать, какие пропущенные значения были обнаружили;
- привести возможные причины появления пропусков в данных;
- объяснить, по какому принципу заполнены пропуски;
- заменить вещественный тип данных на целочисленный:
- пояснить, как выбиран метод для изменения типа данных;
- удалить дубликаты:
- пояснить, как выбиран метод для поиска и удаления дубликатов в данных;
- привести возможные причины появления дубликатов;
- выделить леммы в значениях столбца с целями получения кредита:
- описать, как проводена лемматизация целей кредита;
- категоризировать данные:
- перечислить, какие «словари» выделили для этого набора данных, и объясните, почему.

По резальтату исследования нужно **ответить на вопросы**: 
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**Описание данных** 
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

##  Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd #Импортируем библиотеку Pandas
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df=data
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df.tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


### Вывод

Получаем датасет в 21525 строк. 

В стобцах трудового стажа и ежемесячного дохода есть пропущенные значения. Их число равно, что поддталкивает на мысль о взаимосвязи возникновения пропусков. 

Имеющиеся значения стобца трудового стажа имеют артефакты: отрицательные и завышенные значения (трудовой стаж равный 343937 дней = 942 года). Следует обратиться к разработчикам, предоставившим выгрузку, чтобы разбраться в природе значений. Возможной причиной возникновения артефактов может служить ошибка при расчёте стажа: даты начала и окончания трудовой детяельности могли быть указаны в разных форматах (dd.mm.yyyy - mm.dd.yyyy)


### Шаг 2. Предобработка данных

## Обработка пропусков

Посчитаем число пропусков в столбцах days_employed и total_income

In [4]:
days_employed_na = data['days_employed'].isnull().sum()
print('Число пропусков в стобце "Трудовой стаж" равно {}'.format(days_employed_na))

total_income_na = data['total_income'].isnull().sum()
print('Число пропусков в стобце "Ежемесячный доход" равно {}'.format(days_employed_na))

if days_employed_na == total_income_na:
    print('Число пропусков совпадает')
else:
    print('Число пропусков отличается')

Число пропусков в стобце "Трудовой стаж" равно 2174
Число пропусков в стобце "Ежемесячный доход" равно 2174
Число пропусков совпадает


Проверим связь связь пропусков в столбцах между сосбой:

In [5]:
df.sort_values('total_income').tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21415,0,NaN,54,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с жильем
21423,0,NaN,63,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,сделка с автомобилем
21426,0,NaN,49,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,недвижимость
21432,1,NaN,38,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,NaN,операции с жильем
21463,1,NaN,35,высшее,0,гражданский брак,1,M,сотрудник,0,NaN,на проведение свадьбы
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Наличие взаимосвязи подтвердили: там, где в одном стоблдце встречается NaN, пропуск встречается и в другом. 
Пропуски могут быть связаны с неполнотой данных о заёмщике.
Поскольку для ответа на поставленные в условии вопросы нам не нужна информация о трудовом стаже, заменим пропуски средними значениями. 

Посчитаем, что в среднем уровень благосостояния должен коррелировать с возрастом заёмщика. Проведём группировку по этому признаку:

In [6]:
data['total_income'].groupby(data['dob_years']).mean()

dob_years
0     158337.955712
19    106212.141123
20    126993.295912
21    143629.769631
22    139840.678505
23    138463.153535
24    150446.100901
25    157572.291387
26    157955.848949
27    177110.490144
28    164685.071256
29    169246.438988
30    171970.318639
31    173887.307980
32    167407.477166
33    181604.680340
34    178092.098754
35    177914.045031
36    182428.883307
37    183334.890130
38    168196.928236
39    184250.715208
40    176353.724146
41    177454.879365
42    184710.960430
43    184750.762352
44    177817.361512
45    173600.360807
46    186505.550986
47    169281.635880
48    177231.169491
49    176097.353303
50    165649.381950
51    170219.301449
52    168032.526480
53    167540.434260
54    156215.871072
55    155247.573839
56    161652.131498
57    155360.539014
58    157405.443193
59    154988.350033
60    142604.267812
61    151922.056389
62    148841.527150
63    143097.516441
64    161777.457555
65    151717.488838
66    131718.850062
67    1379

In [7]:
data['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))

А недастающие данные о ежемесячном доходе заменим медианными значениями в зависимости от возраста заёмщика,
чтобы они не исказили результаты последующего анализа:

In [8]:
data['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

Проверим замену пропусков:

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропущенные значения в столбцах ежемесячного заработка и трудового стажа заменили медианными и средними значениями соответственно. 
Вызвав метод .info() получаем, что пропусков в датафрейме больше нет.

Сгруппировал данные в столбцах по возрасту заёмщиков. Думаю, это как раз один из косвенных факторов дохода.

In [10]:
# Код ревьюера
data['total_income'].value_counts()

142594.396847    1105
172357.950966     509
118514.486412     414
150447.935283     147
499163.144947       2
                 ... 
133299.194693       1
115080.782380       1
84896.781597        1
153838.839212       1
150014.128510       1
Name: total_income, Length: 19353, dtype: int64

## Замена типа данных

Чтобы визульно упростить данные и облегчить будущие расчёты, преобразуем данные типа float64 в int64:

In [11]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Избавились от вещественного типа данных (float) в столбцах total_income и days_employed при помощи метода .astype().
Был выбран этот метод, так как .to_numeric() преобразует строковые значения в численные. Получить тип int при его помощи в данном случае не получилось бы.

### Обработка дубликатов

Проверим число дубликатов: 

In [13]:
data.duplicated().sum()

54

Проверим столбцы "gender", "education", "family_status" и "income_type" на уникальные значения:

In [14]:
print(data['gender'].unique()) 
print()
print(data['education'].unique())
print()
print(data['family_status'].unique()) 
print()
print(data['income_type'].unique()) 

['F' 'M' 'XNA']

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Приведём все значения столбца "education" к нижнему регистру:

In [15]:
data['education'] = data['education'].str.lower()

#проверим, как сработала замена, и как это повлияло на число дубликтов:
print(data['education'].unique())
data.duplicated().sum()

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


71

Также упростим значения семейного положения:

In [16]:
data['family_status'] = data['family_status'].replace('Не женат / не замужем', 'не женат')
data['family_status'] = data['family_status'].replace('вдовец / вдова', 'вдовец')
data['family_status'] = data['family_status'].replace('женат / замужем', 'женат')

Избавимся от дубликатов:

In [17]:
data = data.drop_duplicates().reset_index(drop=True)

#проверим, как сработал метод:
data.duplicated().sum()

0

### Вывод

Фрейм содержал дубликаты, часть из которых сперва не удалось распознать методом .duplicated() из-за разного написания значений столбца "education". Приведя их к нижнему регистру, выявили ещё 17 дублирующихся строк и избавились от всех разом, применив .drop_duplicates(). Данные становятся всё краше :)

## Лемматизация

In [18]:
#импортируем pymystem:
from pymystem3 import Mystem
m = Mystem()
data['lemmas'] = data.purpose.map(m.lemmatize) #применяем метод .lemmatize из библиотеки pymystem вместе с .map

In [19]:
def purpose_group(row): #создаем функцию для категоризации лемм
    row_str = str(row)
    lemmas = m.lemmatize(row_str)
    if  'недвижимость'in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль'in lemmas:
        return 'автомобиль'
    if 'свадьба'in lemmas:
        return 'свадьба'
    return 'образование'

data['purpose_category'] = data['lemmas'].apply(purpose_group)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,среднее,1,женат,0,M,сотрудник,0,135823,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

С помощью библиотеки Pymystem и метода lemmatize выделили леммы из слов в столбце Purpose. Сразу разбили причины заявки на кредит на основные категории: недвижимость, автомобиль, образование, свадьба.

## Категоризация данных

Разделим заёмщиков на плательщиков и должников 

In [20]:
def debt_group(debt):
        if debt == 1:
                return 'должник'
        return 'молодец' 
    
data['debt_group'] = data['debt'].apply(debt_group) #применяем функцию к столбцу
print(data['debt_group'].value_counts()) 

молодец    19713
должник     1741
Name: debt_group, dtype: int64


Разделим заёмщиков по уровню дохода

In [21]:
data['income_group'] = pd.qcut(data['total_income'], q=4)
print(data['income_group'].value_counts()) 

(107623.0, 142594.0]      5479
(195820.25, 2265604.0]    5364
(20666.999, 107623.0]     5364
(142594.0, 195820.25]     5247
Name: income_group, dtype: int64


### Вывод по шагу 5

Получили 4 относительно равные группы заёмщиков по уровню дохода при помощи метода .qcut

Сведём число заёмщиков в группах по заработку в словарь:

In [22]:
income_dict = {'средний': 6415,
              'низкий': 5364,
              'высокий': 5364,
              'выше среднего': 4311}

In [23]:
print('Число заёмщиков с низким уровнем дохода равно: {}'.format(income_dict.get('низкий')))

Число заёмщиков с низким уровнем дохода равно: 5364


## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
df = data

def child_debt(df, children, debt_group):
    debt_count = 0
    debt_list = df[(df['children']==children)&(df['debt_group']==debt_group)]
    debt_count = debt_list['debt_group'].count()
    return debt_count

In [25]:
no_child_debt = child_debt(df, 0, 'должник')
one_child_debt = child_debt(df, 1, 'должник')
two_child_debt = child_debt(df, 2, 'должник')
three_child_debt = child_debt(df, 3, 'должник')

In [26]:
one_child = data['children'].groupby(data['children'] == 1).count()
two_child = data['children'].groupby(data['children'] == 2).count()
three_child = data['children'].groupby(data['children'] == 3).count()
no_child = data['children'].groupby(data['children'] == 0).count()

Получили число должников по категориям. Узнаем, какой процент людей является должником в зависимости от наличия детей:

In [27]:
childfree_percent = (child_debt(df, 0, 'должник')) * 100 / no_child
one_percent = (child_debt(df, 1, 'должник')) * 100 / one_child
two_percent = (child_debt(df, 2, 'должник')) * 100 / two_child
three_percent = (child_debt(df, 3, 'должник')) * 100 / three_child


data_child_debt = [['0', no_child_debt, childfree_percent],
       ['1', one_child_debt, one_percent],
       ['2', two_child_debt, two_percent],
       ['3', three_child_debt, three_percent]]

columns = ['Число детей','Число должников','Процент среди категории']
table = pd.DataFrame(data = data_child_debt, columns = columns)
table 

,Число детей,Число должников,Процент среди категории
0,0,1063,children False 14.437050 True 7.543822...
1,1,444,children False 2.667307 True 9.234609 N...
2,2,194,children False 0.999897 True 9.454191 N...
3,3,27,children False 0.127817 True 8.181818 N...


Заёмщики без детей просрачивают возврат долга в 7,54% случаев, а те, у кого есть хотя бы один - в 9,2%. При этом, говорить о прямой зависимости между числом детей и возвратом кредита в срок не приходится - группа заёмщиков с тремя детьми прасрачивает платежи в 8,18% что реже, чем в случае с заёмщиками с 1 и 2 детьми.
Самыми надёжными остаются заёмщики без детей. Это объясняется тем, что родителям чаще тяжелее содержать семью и вовремя оплачивать долги. 

Зависимость присутствует: отсутствие детей при прочих равных характеризует заёмщика как имеющего больше шансов вернуть кредит в срок. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
df.family_status.unique()

array(['женат', 'гражданский брак', 'вдовец', 'в разводе', 'не женат'],
      dtype=object)

In [29]:
def family_status_debt(df, family_status, debt_group):
    debt_count = 0
    debt_list = df[(df['family_status']==family_status)&(df['debt_group']==debt_group)]
    debt_count = debt_list['debt_group'].count()
    return debt_count

In [30]:
income_data = data.pivot_table(index=['family_status'], columns='debt_group', values='debt', aggfunc='count')
print(income_data)

debt_group        должник  молодец
family_status                     
в разводе              85     1110
вдовец                 63      896
гражданский брак      388     3763
женат                 931    11408
не женат              274     2536


In [31]:
income_data = income_data['должник'] * 100 / (income_data['молодец'] + income_data['должник'])
print(income_data)

family_status
в разводе           7.112971
вдовец              6.569343
гражданский брак    9.347145
женат               7.545182
не женат            9.750890
dtype: float64


Чаще всего не возвращают кредиты в срок не женатые и не замужние, а также находящиеся в гражданском браке заёмщики и заёмщицы. Их доля составляет 9,75% и 9,35% соответственно. Далее следуют женатые / замужние, потом - разведённые. Самыми надёжными заёмщиками из данной выборки оказались вдовцы и вдовы.Только 6,6% из них просрачивают платежи. 
Из полученных результатов следует, что прямой зависимости между семейным положением и возвратом кредита в срок нет. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
def income_debt(df, income_group, debt_group):
    debt_count = 0
    debt_list = df[(df['income_group']==income_group)&(df['debt_group']==debt_group)]
    debt_count = debt_list['debt_group'].count()
    return debt_count

Посмотрим, какое число заёмщиков просрачивают выплаты в зависимости от заработка.

In [33]:
income_data = data.pivot_table(index=['income_group'], columns='debt_group', values='debt', aggfunc='count')
display(income_data)

debt_group,должник,молодец
income_group,,
"(20666.999, 107623.0]",427,4937
"(107623.0, 142594.0]",483,4996
"(142594.0, 195820.25]",448,4799
"(195820.25, 2265604.0]",383,4981


In [34]:
income_data = income_data['должник'] * 100 / (income_data['молодец'] + income_data['должник'])
display(income_data)

income_group
(20666.999, 107623.0]     7.960477
(107623.0, 142594.0]      8.815477
(142594.0, 195820.25]     8.538212
(195820.25, 2265604.0]    7.140194
dtype: float64

Получаем, что наиболее надёжными являются обеспеченные граждане, зарабатывающие от 195 000 рублей. При этом полной зависимости между достатком и возвратом кредита в срок нет. Следом за самыми обеспеченными идут самые необеспеченные. Больше всего должников среди средней категории, зарабывотывающей от 107 до 195 тысяч рублей. 

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
def purpose_debt(df, purpose_category, debt_group):
    debt_count = 0
    debt_list = df[(df['purpose_category']==purpose_category)&(df['debt_group']==debt_group)]
    debt_count = debt_list['debt_group'].count()
    return debt_count

Узнаем, какое число людей является должником в зависимости от причины:

In [36]:
#purposes_all = data['purpose_category'].value_counts()
purposes_all = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='count')
display(purposes_all)

,debt
purpose_category,
автомобиль,4306
недвижимость,10811
образование,4013
свадьба,2324


In [37]:
purpose_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='sum')
display(purpose_pivot)

,debt
purpose_category,
автомобиль,403
недвижимость,782
образование,370
свадьба,186


In [38]:
purposes_percent = purpose_pivot * 100 / purposes_all
display(purposes_percent)

,debt
purpose_category,
автомобиль,9.359034
недвижимость,7.233373
образование,9.220035
свадьба,8.003442


В результате видим, что наивысший процент неплательщиков есть среди тех заёмщиков, что берут кредит на покупку автомобиля и образование. Самыми порядочными являются заёмщики, совершающие операции с недвижимостью. 

Это можно объяснить тем, что кредит на образование чаще берут молодые люди без стабильных источников заработка, а автолюбители склонны завышать потребности, приобретая более дорогие автомобили, чем могут себе позволить. 

##  Общий вывод

Таким образом мы подтвердили одну из трёх гипотез по определению взаимосвязи, выявив зависимость возврата кредита в срок от числа детей в семье заёмщика. Семейное положение не оказало существенного видимого влияния на исследуемый параментр. 

Что касается цели кредита, наиболее рисковыми для банка являются автокредитование и кредиты на образование. Радует, что число заёмщиков, берущих кредиты на свадьбу, наименьшее из выборки, при этом 8% из них также просрачивают платежи. Самая надёжная для банка сфера из представленных - ипотечное кредитование. 

Идеальный банковский заёмщик: бездетный, разведённый, обеспеченный, берущий кредит на покупку недвижимости. 